# W261 Final Project ETL for Development Sample

### Notebook Set-Up

In [10]:
# imports
import time
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
#mllib.linalg library 
from pyspark.sql import DataFrame

In [11]:
%reload_ext autoreload
%autoreload 2

In [12]:
# store path to notebook
#PWD = !pwd
#PWD = PWD[0]

In [13]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Load Data

In [14]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

## Transformation

### Sample

In [15]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.9999,0.0001], seed = 1)
sampleRDD2.cache()

PythonRDD[8] at RDD at PythonRDD.scala:49

In [8]:
nrow = sampleRDD2.count()
print("This sample contains", str(nrow), "rows.")

This sample contains 4478 rows.


In [16]:
sample_testRDD1, sample_testRDD2 = original_trainRDD.randomSplit([0.9999,0.000025], seed = 1)
sample_testRDD2
sample_testRDD2.count()

1098

# Put in wide, sparse feature format

In [17]:
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])  ## y variable 
    
    # ignore numerics to start
    #numerical_values = list(pd.Series(raw_values[1:14]).apply(pd.to_numeric))
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)


def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features")
    
    model = cv.fit(DF)
    result = model.transform(DF)
    
    return result
parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [18]:
#not sure why its 30,946 in the first column...comes out of countvectorizor on spark
#https://spark.apache.org/docs/latest/ml-features.html#countvectorizer
vectorizedDF = vectorizeCV(parsedDF)
vectorizedDF.show()

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(30946,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(30946,[0,1,2,5,1...|
|    1|[n1_1, n2_1, n4_9...|(30946,[0,1,6,7,1...|
|    0|[n0_8, n1_17, n3_...|(30946,[0,1,4,12,...|
|    1|[n0_6, n1_1, n2_7...|(30946,[0,1,2,4,1...|
|    1|[n1_99, n2_1, n3_...|(30946,[1,2,4,10,...|
|    0|[n0_3, n1_21, n2_...|(30946,[0,1,4,8,1...|
|    0|[n1_2, n2_20, n3_...|(30946,[0,1,3,5,8...|
|    0|[n0_0, n1_144, n4...|(30946,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(30946,[0,2,3,6,1...|
|    0|[n0_0, n1_1, n2_4...|(30946,[0,1,2,3,5...|
|    0|[n0_9, n1_5, n2_1...|(30946,[0,2,3,6,9...|
|    0|[n1_323, n2_2, n3...|(30946,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(30946,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(30946,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(30946,[1,2,8,14,...|
|    0|[n1_126, n2_2, n3...|(30946,[0,2,4,6,8...|


In [19]:
parsedDF = sample_testRDD2.map(parseCV).toDF().cache()
vectorizedTest = vectorizeCV(parsedDF)
vectorizedTest = vectorizeCV(parsedDF)
vectorizedTest.show()

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(10968,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(10968,[0,1,2,5,8...|
|    0|[n1_0, n2_5, n4_3...|(10968,[0,2,3,6,1...|
|    0|[n1_323, n2_2, n3...|(10968,[1,2,13,16...|
|    0|[n0_0, n1_13, n2_...|(10968,[0,1,2,5,6...|
|    1|[n1_4, n2_1, n3_2...|(10968,[0,6,7,15,...|
|    1|[n0_2, n1_0, n2_1...|(10968,[0,1,2,4,1...|
|    0|[n0_0, n1_23, n2_...|(10968,[0,1,3,7,1...|
|    0|[n0_0, n1_15, n2_...|(10968,[0,2,3,4,1...|
|    0|[n1_54, n2_1, n3_...|(10968,[0,1,2,3,4...|
|    0|[n0_0, n1_-1, n2_...|(10968,[0,1,4,5,7...|
|    0|[n1_3, n2_2, n4_4...|(10968,[0,1,2,3,5...|
|    0|[n0_2, n1_180, n2...|(10968,[0,1,2,3,4...|
|    0|[n0_0, n1_1, n2_2...|(10968,[0,2,3,4,5...|
|    0|[n1_3, n2_8, n3_8...|(10968,[5,8,13,16...|
|    1|[n0_8, n1_-1, n3_...|(10968,[0,1,2,3,4...|
|    0|[n1_4, n4_8012, n...|(10968,[0,1,13,15...|


In [20]:
sc.addPyFile("fm_parallel_sgd.py")
import fm_parallel_sgd as fm

In [21]:
print (vectorizedDF.rdd.count())
print (vectorizedTest.rdd.count())
print (vectorizedDF.rdd.first())

4478
1098
Row(label=0, raw=['n1_4', 'n2_50', 'n3_18', 'n4_3339', 'n5_20', 'n6_26', 'n7_17', 'n8_133', 'n10_2', 'n12_18', 'c0_09ca0b81', 'c1_09e68b86', 'c2_86c4b829', 'c3_e3d0459f', 'c4_25c83c98', 'c6_7227c706', 'c7_0b153874', 'c8_a73ee510', 'c9_305a0646', 'c10_9625b211', 'c11_997a695a', 'c12_dccbd94b', 'c13_07d13a8f', 'c14_36721ddc', 'c15_c0b906bb', 'c16_e5ba7672', 'c17_5aed7436', 'c18_21ddcdc9', 'c19_a458ea53', 'c20_0cbbcc92', 'c22_32c7478e', 'c23_0174dd24', 'c24_3d2bedd7', 'c25_d8ecbc17'], features=SparseVector(30946, {0: 1.0, 1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 20: 1.0, 32: 1.0, 122: 1.0, 154: 1.0, 174: 1.0, 214: 1.0, 365: 1.0, 369: 1.0, 495: 1.0, 504: 1.0, 632: 1.0, 635: 1.0, 877: 1.0, 1902: 1.0, 2124: 1.0, 2252: 1.0, 2393: 1.0, 2791: 1.0, 6288: 1.0, 10030: 1.0, 13371: 1.0, 14494: 1.0, 18220: 1.0, 19944: 1.0, 24008: 1.0, 25592: 1.0, 26671: 1.0}))


In [22]:
temp = time.time()
model = fm.trainFM_parallel_sgd (sc, vectorizedDF.rdd, iterations=1, iter_sgd= 1, alpha=0.01, regParam=0.01, factorLength=4,\
                      verbose=True, savingFilename = None, evalTraining=None)
print ('time :', time.time()-temp)

iter 	time 	train_logl 	val_logl
0 	0 	0.696410 	0.696206


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 217 in stage 32.0 failed 1 times, most recent failure: Lost task 217.0 in stage 32.0 (TID 6561, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1848)
	at java.io.ObjectOutputStream.writeArray(ObjectOutputStream.java:1334)
	at java.io.ObjectOutputStream.writeObject0(Ob

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 36800)
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
---------------------------------

In [ ]:
print (evaluate(vectorizedTest, model))